In [ ]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine
!pip install pickle5

In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *
import pickle5 as pickle

In [ ]:
df_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson')
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [229]:
# 우리동네 키움포털 
df_crawl = pd.read_csv('/content/drive/My Drive/Project/Seoul/돌봄시설 api/우리동네키움포털.csv').drop('Unnamed: 0', axis=1)
df_crawl

,센터,주소,상시돌봄,일시돌봄
0,강남1호점 우리동네키움센터강남 다함께키움센터태화,서울특별시 강남구 광평로 185,45,3
1,강남2호점 우리동네키움센터강남 다함께키움센터역삼,서울특별시 강남구 논현로64길 7,20,4
2,강남3호점 우리동네키움센터강남 다함께키움센터청담,서울특별시 강남구 영동대로131길 26,20,4
3,강남4호점 우리동네키움센터미미위 세곡 키움센터,서울특별시 강남구 밤고개로 231,25,5
4,강동1호점 우리동네키움센터상상나루來키움센터,서울특별시 강동구 올림픽로98가길 28,20,4
...,...,...,...,...
86,중구2호점 우리동네키움센터모든아이 중림센터,서울특별시 중구 서소문로6길 16,40,8
87,중구3호점 우리동네키움센터모든아이 손기정센터,서울특별시 중구 손기정로 101,40,8
88,중구4호점 우리동네키움센터모든아이 약수센터,서울특별시 중구 동호로5길 19,80,16
89,중구5호점 우리동네키움센터모든아이 신당5동센터,서울특별시 중구 다산로44길 85,30,6


In [ ]:
# 돌봄센터 ID 만들기 

df_dolbom['시설ID'] = df_dolbom['시설명'].map(lambda x: x.split(' ')[0])
df_dolbom['시설ID'].unique()
df_dolbom = df_dolbom[['자치구', '행정동코드', '법정동', '행정동', '시설ID','시설명', '주소', 'Latitude', 'Longitude',
       'geometry']]
df_dolbom.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson', driver='GeoJSON')
df_dolbom.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.csv', index=False)

In [ ]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(school_loc, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(school_loc['학교명'],school_loc['위도'],school_loc['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

## 0. 학교데이터로 Target 만들기

In [ ]:
# 학교데이터 불러오기 
with open('/content/drive/My Drive/Project/Seoul/초등학교/초중고위치.pkl', 'rb') as f:  
  df_school = pickle.load(f)

#### 학교데이터 전처리 

In [ ]:
mask = df_school['학교급구분'].str.contains('초등') 
df_school = df_school[mask].reset_index().drop(['index'], axis=1)

In [ ]:
df_school['위도'] = df_school['위도'].astype(float)
df_school['경도'] = df_school['경도'].astype(float)

# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_school['경도'], df_school['위도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
df_school = gpd.GeoDataFrame(df_school, crs=crs, geometry=geometry)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
df_school = df_school.drop('동',axis=1)
df_school

,학교명,학교급구분,소재지지번주소,소재지도로명주소,위도,경도,구,법정동코드,법정동,행정동코드,행정동,geometry
0,서울휘경초등학교,초등학교,서울특별시 동대문구 휘경동 302-2,서울특별시 동대문구 망우로6길 48,37.585736,127.058213,동대문구,1123010900,휘경동.1,1123073000,휘경제2동,POINT (127.05821 37.58574)
1,서울동답초등학교,초등학교,서울특별시 동대문구 답십리동 2-24,서울특별시 동대문구 답십리로60길 12,37.571503,127.064034,동대문구,1123010500,답십리동,1123061000,답십리제2동,POINT (127.06403 37.57150)
2,서울답십리초등학교,초등학교,서울특별시 동대문구 답십리동 274,서울특별시 동대문구 전농로3길 23,37.568804,127.055429,동대문구,1123010500,답십리동,1123060000,답십리제1동,POINT (127.05543 37.56880)
3,서울신답초등학교,초등학교,서울특별시 동대문구 답십리동 463-13,서울특별시 동대문구 천호대로 177,37.572705,127.042962,동대문구,1123010500,답십리동,1123060000,답십리제1동,POINT (127.04296 37.57271)
4,서울용두초등학교,초등학교,서울특별시 동대문구 용두동 237-1,서울특별시 동대문구 천호대로13길 43,37.576415,127.028387,동대문구,1123010200,용두동,1123053600,용신동,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...,...,...,...,...
598,서울위례별초등학교,초등학교,서울특별시 송파구 장지동 886,서울특별시 송파구 위례광장로 243,37.479086,127.145042,송파구,1171010900,장지동,1171064700,위례동,POINT (127.14504 37.47909)
599,서울하늘숲초등학교,초등학교,서울특별시 구로구 천왕동 산13-1,서울특별시 구로구 오리로 1100,37.483405,126.842135,구로구,1153011100,천왕동,1153078000,오류제2동,POINT (126.84213 37.48340)
600,서울언남초등학교,초등학교,서울특별시 서초구 내곡동 121,서울특별시 서초구 헌릉로8길 42,37.453898,127.061885,서초구,1165010900,내곡동,1165066000,내곡동,POINT (127.06189 37.45390)
601,서울항동초등학교,초등학교,서울특별시 구로구 항동 160,서울특별시 구로구 연동로 178,37.476972,126.824078,구로구,1153011200,항동,1153080000,항동,POINT (126.82408 37.47697)


In [ ]:
df_school.to_pickle('/content/drive/My Drive/Project/Seoul/초등학교/초등학교.pickle')

### 학교데이터로 Target 만들기

In [ ]:
taregt_school = df_school[['행정동코드','구', '행정동','학교명','소재지도로명주소', '위도', '경도', 'geometry']]
taregt_school

,행정동코드,구,행정동,학교명,소재지도로명주소,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,37.585736,127.058213,POINT (127.05821 37.58574)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,37.571503,127.064034,POINT (127.06403 37.57150)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,37.568804,127.055429,POINT (127.05543 37.56880)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,37.572705,127.042962,POINT (127.04296 37.57271)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,37.576415,127.028387,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,37.479086,127.145042,POINT (127.14504 37.47909)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,37.483405,126.842135,POINT (126.84213 37.48340)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,37.453898,127.061885,POINT (127.06189 37.45390)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,37.476972,126.824078,POINT (126.82408 37.47697)


In [ ]:
merge_base = taregt_school[['학교명']]
merge_base

,학교명
0,서울휘경초등학교
1,서울동답초등학교
2,서울답십리초등학교
3,서울신답초등학교
4,서울용두초등학교
...,...
598,서울위례별초등학교
599,서울하늘숲초등학교
600,서울언남초등학교
601,서울항동초등학교


## 1. 우리마을키움센터

#### 전처리

In [241]:
df_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson')
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [242]:
# 우리동네 키움포털 
df_crawl = pd.read_csv('/content/drive/My Drive/Project/Seoul/돌봄시설 api/우리동네키움포털.csv').drop(['Unnamed: 0','주소'], axis=1)
df_crawl['시설ID'] = df_crawl['센터'].map(lambda x: x.split(' ')[0])
df_crawl

,센터,상시돌봄,일시돌봄,시설ID
0,강남1호점 우리동네키움센터강남 다함께키움센터태화,45,3,강남1호점
1,강남2호점 우리동네키움센터강남 다함께키움센터역삼,20,4,강남2호점
2,강남3호점 우리동네키움센터강남 다함께키움센터청담,20,4,강남3호점
3,강남4호점 우리동네키움센터미미위 세곡 키움센터,25,5,강남4호점
4,강동1호점 우리동네키움센터상상나루來키움센터,20,4,강동1호점
...,...,...,...,...
86,중구2호점 우리동네키움센터모든아이 중림센터,40,8,중구2호점
87,중구3호점 우리동네키움센터모든아이 손기정센터,40,8,중구3호점
88,중구4호점 우리동네키움센터모든아이 약수센터,80,16,중구4호점
89,중구5호점 우리동네키움센터모든아이 신당5동센터,30,6,중구5호점


In [232]:
# # 돌봄센터 ID 만들기 

# df_dolbom['시설ID'] = df_dolbom['시설명'].map(lambda x: x.split(' ')[0])
# df_dolbom['시설ID'].unique()
# df_dolbom = df_dolbom[['자치구', '행정동코드', '법정동', '행정동', '시설ID','시설명', '주소', 'Latitude', 'Longitude',
#        'geometry']]
# df_dolbom.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson', driver='GeoJSON')
# df_dolbom.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.csv', index=False)

In [243]:
df_merge = df_dolbom.merge(df_crawl, how ='left', on ='시설ID').drop('센터', axis=1)
df_dolbom = df_merge[['자치구', '행정동코드', '법정동', '행정동', '시설ID', '시설명', '상시돌봄', '일시돌봄','주소', 'Latitude',
       'Longitude', 'geometry']]
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [244]:
# df_dolbom.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson', driver='GeoJSON')
# df_dolbom.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.csv', index=False)

### 거리

In [245]:
# Point로 나와 있는 데이터들 LSCP 함수 
# 아아들 도보 10분:  749.82m ~ 880.98m -> 오차 고려해서 881m로 

def harv_dis(school_loc, target, dist):
  lscp=[]
  target_nm=target.columns.tolist()[[(t.find('시설ID')>=0) | (t.lower().find('시설명')>=0) | 
                                     (t.lower().find('parking_nm')>=0) | (t.lower().find('place_nm')>=0)|
                                     (t.find('시설명')>=0) for t in target.columns.tolist()].index(True)]

  for school_nm, lat, long in tqdm(zip(school_loc['학교명'],school_loc['위도'],school_loc['경도'])):
    try:
      for nm, t_lat, t_long in zip(target[target_nm],target['Latitude'],target['Longitude']):
        t_lat=float(t_lat)
        t_long=float(t_long)
        x = (lat, long)
        y = (t_lat, t_long) 
        h = haversine(x, y, unit='m')
        if h <=dist:
          lscp.append([school_nm,nm,h])
    except:
      try:
        for nm, t_lat, t_long, t_count in zip(target[target_nm],target['Y'],target['X'],target['count']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,t_count,h])
      except:
        for nm, t_lat, t_long in zip(target[target_nm],target['Y'],target['X']):
          t_lat=float(t_lat)
          t_long=float(t_long)
          x = (lat, long)
          y = (t_lat, t_long) 
          h = haversine(x, y, unit='m')
          if h <=dist:
            lscp.append([school_nm,nm,t_lat,t_long,h])
  return lscp

In [246]:
df_dolbom = gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/돌봄시설_위치_행정동.geojson')
df_dolbom

,자치구,행정동코드,법정동,행정동,시설ID,시설명,상시돌봄,일시돌봄,주소,Latitude,Longitude,geometry
0,중랑구,1126068000,신내동,신내1동,중랑1호점,중랑1호점 신내동우리동네키움센터(중랑아동돌봄센터),22,4,서울특별시 중랑구 신내역로1길 145,37.617108,127.106766,POINT (127.10677 37.61711)
1,중구,1114065000,신당동,신당제5동,중구5호점,중구5호점 우리동네키움센터(모든아이 신당5동센터),30,6,서울특별시 중구 다산로44길 85,37.565088,127.021852,POINT (127.02185 37.56509)
2,중구,1114063500,신당동,약수동,중구4호점,중구4호점 우리동네키움센터(모든아이 약수센터),80,16,서울특별시 중구 동호로5길 19,37.551780,127.012577,POINT (127.01258 37.55178)
3,중구,1114068000,만리동1가,중림동,중구3호점,중구3호점 우리동네키움센터(모든아이 손기정센터),40,8,서울특별시 중구 손기정로 101,37.555665,126.964550,POINT (126.96455 37.55567)
4,중구,1138058000,중림동,응암제1동,중구2호점,중구2호점 우리동네키움센터(모든아이 중림센터),40,8,서울특별시 중구 서소문로6길 16,37.598391,126.918197,POINT (126.91820 37.59839)
...,...,...,...,...,...,...,...,...,...,...,...,...
86,강동구,1174057000,암사동,암사제1동,강동1호점,강동1호점 우리동네키움센터(상상나루來키움센터),20,4,서울특별시 강동구 올림픽로98가길 28,37.550551,127.131430,POINT (127.13143 37.55055)
87,강남구,1168070000,자곡동,세곡동,강남4호점,강남4호점 우리동네키움센터(미미위 세곡 키움센터),25,5,서울특별시 강남구 밤고개로 231,37.473781,127.106570,POINT (127.10657 37.47378)
88,강남구,1168056500,청담동,청담동,강남3호점,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),20,4,서울특별시 강남구 영동대로131길 26,37.520398,127.054127,POINT (127.05413 37.52040)
89,강남구,1168065000,역삼동,역삼2동,강남2호점,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),20,4,서울특별시 강남구 논현로64길 7,37.493925,127.040900,POINT (127.04090 37.49393)


In [247]:
lscp_dolbom=pd.DataFrame(harv_dis(taregt_school,df_dolbom,881), columns=['학교명','돌봄센터','돌봄센터_거리'])
lscp_dolbom

,학교명,돌봄센터,돌봄센터_거리
0,서울동답초등학교,동대문2호점,639.866446
1,서울신답초등학교,성동8호점,844.669250
2,서울신답초등학교,성동3호점,506.946984
3,서울용두초등학교,성동8호점,817.863932
4,서울용두초등학교,동대문1호점,172.838994
...,...,...,...
352,서울하늘숲초등학교,구로5호점,521.321341
353,서울숭신초등학교,중구5호점,612.779818
354,서울숭신초등학교,성동8호점,598.008884
355,서울숭신초등학교,성동2호점,816.383363


In [248]:
# 돌봄센터 하나에 여러 학교 
lscp_dolbom.groupby(['돌봄센터','학교명']).agg('mean')

돌봄센터_거리
돌봄센터  학교명                 
강남1호점 서울대모초등학교  801.467851
      서울왕북초등학교  673.274058
강남2호점 서울역삼초등학교  732.487180
강남3호점 서울봉은초등학교  601.073326
      서울언북초등학교  830.713113
...                    ...
중구5호점 서울무학초등학교  713.176809
      서울숭신초등학교  612.779818
      서울신당초등학교  228.519713
      서울흥인초등학교  796.039353
중랑1호점 서울새솔초등학교  413.691542

[357 rows x 1 columns]

In [249]:
# 학교 하나에 여러 돌봄센터 
lscp_dolbom.groupby(['학교명', '돌봄센터']).agg('mean')

돌봄센터_거리
학교명    돌봄센터              
광운초등학교 노원1호점   396.628858
       노원4호점   404.378192
       성북1호점   787.214751
대광초등학교 동대문1호점  694.076713
동산초등학교 중구1호점   608.415921
...                   ...
청원초등학교 노원3호점   439.522567
       노원9호점   737.410728
충암초등학교 은평3호점   640.953500
한양초등학교 성동9호점   554.853600
화랑초등학교 노원6호점   787.040932

[357 rows x 1 columns]

In [250]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_dolbom, columns = ['학교명','돌봄센터','돌봄센터_거리']) 
lscp_dolbom_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_dolbom_loc = pd.DataFrame(lscp_dolbom_groupby.items())
df_dolbom_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 돌봄센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_dolbom_groupby.keys()):
  stations.append(lscp_dolbom_groupby[k]['돌봄센터'].values)

#초등학교와 각 초등학교별 돌봄센터 데이터 프레임 합치기 
df_dolbom_loc['돌봄센터'] = stations
df_dolbom_loc = df_dolbom_loc[[0, '돌봄센터']]
df_dolbom_loc.columns = ['학교명', '돌봄센터']
df_dolbom_loc

,학교명,돌봄센터
0,광운초등학교,"[성북1호점, 노원4호점, 노원1호점]"
1,대광초등학교,[동대문1호점]
2,동산초등학교,"[중구4호점, 중구1호점]"
3,명지초등학교,[은평3호점]
4,상명초등학교,"[노원5호점, 노원19호점, 노원18호점, 노원17호점]"
...,...,...
226,은석초등학교,[동대문2호점]
227,청원초등학교,"[노원9호점, 노원3호점, 노원2호점, 노원22호점, 노원21호점, 노원16호점]"
228,충암초등학교,[은평3호점]
229,한양초등학교,[성동9호점]


In [251]:
df_dolbom_loc = merge_base.merge(df_dolbom_loc,on='학교명', how='outer')
df_dolbom_loc

,학교명,돌봄센터
0,서울휘경초등학교,NaN
1,서울동답초등학교,[동대문2호점]
2,서울답십리초등학교,NaN
3,서울신답초등학교,"[성동8호점, 성동3호점]"
4,서울용두초등학교,"[성동8호점, 동대문1호점]"
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,[구로5호점]
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


## 초등돌봄센터

In [252]:
element = pd.read_excel("/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.xlsx")
element

,시군구,학교명,교실수,이용인원,소재지
0,종로구,서울교동초등학교,3,50,서울특별시 종로구 삼일대로 446 (경운동)
1,종로구,서울독립문초등학교,4,80,서울특별시 종로구 통일로12길 23 (무악동)
2,종로구,서울매동초등학교,3,65,서울특별시 종로구 사직로9길 19 . 서울매동초등학교 (필운동)
3,종로구,서울명신초등학교,3,60,서울특별시 종로구 낙산길 250 (창신동)
4,종로구,서울세검정초등학교,5,111,서울특별시 종로구 세검정로9길 1 (신영동.서울세검정초등학교)
...,...,...,...,...,...
567,강동구,서울신암초등학교,4,100,서울특별시 강동구 올림픽로93길 33-16 . 서울신암초등학교 (암사동)
568,강동구,서울천동초등학교,4,89,서울특별시 강동구 성안로 200 . 서울천동초등학교 (길동)
569,강동구,서울천일초등학교,2,45,서울특별시 강동구 구천면로 266 . 서울천일초등학교 (천호동)
570,강동구,서울천호초등학교,3,74,서울특별시 강동구 구천면로 310-11 (천호동.서울천호초등학교)


#### 초등돌봄 전처리 

In [256]:
element = pd.read_excel("/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx")
element

,시군구,학교명,교실수,이용인원,소재지
0,종로구,서울교동초등학교,3,50,서울특별시 종로구 삼일대로 446 (경운동)
1,종로구,서울독립문초등학교,4,80,서울특별시 종로구 통일로12길 23 (무악동)
2,종로구,서울매동초등학교,3,65,서울특별시 종로구 사직로9길 19 . 서울매동초등학교 (필운동)
3,종로구,서울명신초등학교,3,60,서울특별시 종로구 낙산길 250 (창신동)
4,종로구,서울세검정초등학교,5,111,서울특별시 종로구 세검정로9길 1 (신영동.서울세검정초등학교)
...,...,...,...,...,...
567,강동구,서울신암초등학교,4,100,서울특별시 강동구 올림픽로93길 33-16 . 서울신암초등학교 (암사동)
568,강동구,서울천동초등학교,4,89,서울특별시 강동구 성안로 200 . 서울천동초등학교 (길동)
569,강동구,서울천일초등학교,2,45,서울특별시 강동구 구천면로 266 . 서울천일초등학교 (천호동)
570,강동구,서울천호초등학교,3,74,서울특별시 강동구 구천면로 310-11 (천호동.서울천호초등학교)


In [255]:
## 초등돌봄시설 전처리 
element = pd.read_excel("/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx")
# element = element.drop(['연번', '시도', '비고'], axis = 1)

# 서울염강초등학교 폐지 후 서울가양초등학교 및 서울염경초등학교로 통･폐합
mask = element['학교명'].str.contains('염강') 
element = element[~mask].reset_index().drop(['index'], axis=1)

# element.to_excel('/content/drive/My Drive/Project/Seoul/초등학교/초등돌봄교실이용현황_서울.xlsx', index = False)

In [254]:
len(element['학교명'].unique())

572

In [257]:
# 서울염강초등학교 폐지 후 서울가양초등학교 및 서울염경초등학교로 통･폐합
for e in element['학교명']:
  if e not in taregt_school['학교명'].values:
    print(e)

In [258]:
taregt_school[taregt_school['학교명'].str.contains('염강')]

,행정동코드,구,행정동,학교명,소재지도로명주소,위도,경도,geometry


#### 위치코드와 합치기 

In [259]:
df_element = taregt_school.merge(element, on='학교명',how='left').drop(['소재지','시군구'], axis=1).fillna(0)
df_element = df_element[['행정동코드', '구', '행정동', '학교명', '소재지도로명주소','교실수', '이용인원', '위도', '경도', 'geometry']]
df_element

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (127.05821 37.58574)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (127.06403 37.57150)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (127.05543 37.56880)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (127.04296 37.57271)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (127.02839 37.57641)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (127.14504 37.47909)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (126.84213 37.48340)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (127.06189 37.45390)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (126.82408 37.47697)


In [260]:
df_element[df_element['이용인원'].isnull()]

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry


In [261]:
# 결측치 0으로 넣었음 
df_element[df_element['이용인원']==0]

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
42,1126069000,중랑구,신내2동,금성초등학교,서울특별시 중랑구 신내로21길 55,0.0,0.0,37.615314,127.089859,POINT (127.08986 37.61531)
70,1138055200,은평구,갈현제2동,선일초등학교,서울특별시 은평구 갈현로29길 27,0.0,0.0,37.619716,126.912800,POINT (126.91280 37.61972)
71,1138056000,은평구,구산동,예일초등학교,서울특별시 은평구 연서로 117,0.0,0.0,37.609642,126.916096,POINT (126.91610 37.60964)
72,1138052000,은평구,불광제1동,은혜초등학교,서울특별시 은평구 통일로78가길 20,0.0,0.0,37.617733,126.927400,POINT (126.92740 37.61773)
87,1141056500,서대문구,충현동,경기초등학교,서울특별시 서대문구 경기대로9길 10,0.0,0.0,37.565110,126.962861,POINT (126.96286 37.56511)
88,1141058500,서대문구,신촌동,이화여자대학교사범대학부속초등학교,서울특별시 서대문구 성산로 512-39,0.0,0.0,37.561149,126.942750,POINT (126.94275 37.56115)
89,1141056500,서대문구,충현동,추계초등학교,서울특별시 서대문구 북아현로11가길 7,0.0,0.0,37.563150,126.953209,POINT (126.95321 37.56315)
90,1141068500,서대문구,홍은제2동,명지초등학교,서울특별시 서대문구 명지2길 32-20,0.0,0.0,37.583630,126.928230,POINT (126.92823 37.58363)
153,1154568000,금천구,시흥제2동,동광초등학교,서울특별시 금천구 탑골로10길 45,0.0,0.0,37.451672,126.916887,POINT (126.91689 37.45167)
242,1111064000,종로구,이화동,서울대학교사범대학부설초등학교,서울특별시 종로구 대학로 64,0.0,0.0,37.577600,127.004297,POINT (127.00430 37.57760)


In [262]:
df_element.to_excel('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.xlsx', index = False)

In [263]:
df_element = df_element.drop(['geometry'], axis=1)
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_element['위도'], df_element['경도'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_element = gpd.GeoDataFrame(df_element, crs=crs, geometry=geometry)
gdf_element

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824)


In [278]:
gdf_element.columns = ['행정동코드', '구', '행정동', '학교명', '소재지도로명주소', '돌봄교실_개수', '돌봄교실_이용인원', '위도', '경도',
       'geometry']
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824)


In [191]:
gdf_element.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson', driver='GeoJSON')

## 지역아동센터

In [270]:
gpd.read_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson')

,행정동코드,구,행정동,학교명,소재지도로명주소,교실수,이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476971,126.824078,POINT (37.477 126.824)


In [264]:
df_local = pd.read_excel('/content/drive/My Drive/Project/Seoul/초등학교/지역아동센터_191231기준.xlsx')
df_local

,시도,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude
0,서울,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319
1,서울,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336
2,서울,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640
3,서울,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204
4,서울,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528
...,...,...,...,...,...,...,...
451,서울,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897
452,서울,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245
453,서울,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566
454,서울,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241


#### 전처리

In [265]:
df_local = df_local.drop(['시도'],axis = 1)
df_local

,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528
...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241


In [266]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [267]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_local['Longitude'], df_local['Latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_local = gpd.GeoDataFrame(df_local, crs=crs, geometry=geometry)
gdf_local

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312)
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730)
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776)
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509)
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528,POINT (127.01653 37.57785)
...,...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537)
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055)
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823)
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207)


In [268]:
# 행정동 포함여부 확인 
gdf_local['행정동코드'] = np.zeros(len(gdf_local))
gdf_local['행정동'] = np.zeros(len(gdf_local))

for i in gdf_local.index:
    for j in dong_geo.index:
        if gdf_local.geometry[i].within(dong_geo.geometry[j]):
            gdf_local['행정동'][i] = dong_geo.읍면동명[j]
            gdf_local['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_local['행정동코드'] = gdf_local['행정동코드'].astype(int)
gdf_local

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,시군구,시설명,입소정원(명),시설 소재지,Latitude,Longitude,geometry,행정동코드,행정동
0,종로구,참신나는학교지역아동센터,35,서울특별시 종로구 종로53길 7-10 (창신동),37.573121,127.013319,POINT (127.01332 37.57312),1111067000,창신제1동
1,종로구,청암지역아동센터,19,서울특별시 종로구 창신8길 13 (창신동),37.577295,127.010336,POINT (127.01034 37.57730),1111068000,창신제2동
2,종로구,해송지역아동센터,29,서울특별시 종로구 낙산성곽동길 59-10 (창신동),37.577762,127.008640,POINT (127.00864 37.57776),1111064000,이화동
3,종로구,성터지역아동센터,19,서울특별시 종로구 창신7가길 14-7 (창신동),37.575094,127.009204,POINT (127.00920 37.57509),1111068000,창신제2동
4,종로구,푸른학교지역아동센터,29,서울특별시 종로구 지봉로12길 38 (숭인동),37.577853,127.016528,POINT (127.01653 37.57785),1111070000,숭인제1동
...,...,...,...,...,...,...,...,...,...
451,강동구,서울중심지역아동센터,43,"서울특별시 강동구 천호대로162길 34, 215호 (성내동)",37.535367,127.127897,POINT (127.12790 37.53537),1174065000,성내제2동
452,강동구,숲과나무지역아동센터,35,서울특별시 강동구 올림픽로98길,37.550553,127.129245,POINT (127.12925 37.55055),1174057000,암사제1동
453,강동구,마을학교지역아동센터,25,"서울특별시 강동구 풍성로54길 27, 3층 (성내동)",37.528230,127.133566,POINT (127.13357 37.52823),1174066000,성내제3동
454,강동구,구립함께하는지역아동센터,19,"서울특별시 강동구 구천면로 656, 4층 (상일동)",37.552071,127.182241,POINT (127.18224 37.55207),0,0


In [ ]:
# gdf_element.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson', driver='GeoJSON')

### 800m 초등학교

In [203]:
lscp_local=pd.DataFrame(harv_dis(taregt_school,df_local,881), columns=['학교명','지역아동센터','돌봄센터_거리'])
lscp_local

,학교명,지역아동센터,돌봄센터_거리
0,서울휘경초등학교,염광지역아동센터빛나는집,850.813117
1,서울휘경초등학교,해피아트지역아동센터,266.110861
2,서울휘경초등학교,원광지역아동센터,722.309281
3,서울휘경초등학교,나누리지역아동센터,824.852274
4,서울동답초등학교,염광지역아동센터빛나는집,819.583702
...,...,...,...
1682,서울하늘숲초등학교,구로구공립지역아동센터,475.358542
1683,서울숭신초등학교,엘림지역아동센터,863.171727
1684,서울숭신초등학교,조이스터디,669.820637
1685,서울숭신초등학교,성동지역아동복지센터,807.600973


In [204]:
# 돌봄센터 하나에 여러 학교 
lscp_local.groupby(['지역아동센터','학교명']).agg('mean')

돌봄센터_거리
지역아동센터           학교명                 
1318해피존새움터지역아동센터 서울구남초등학교  837.289475
                 서울성자초등학교  339.970343
                 서울신자초등학교  833.378772
                 서울양남초등학교  643.584364
                 서울자양초등학교  418.287768
...                               ...
희망의집지역아동센터       서울수유초등학교  361.894776
                 서울유현초등학교  737.416801
희망지역아동센터         서울신강초등학교  847.963519
                 서울신정초등학교  531.971856
                 서울양동초등학교  877.255376

[1687 rows x 1 columns]

In [205]:
# 학교 하나에 여러 돌봄센터 
lscp_local.groupby(['학교명', '지역아동센터']).agg('mean')

돌봄센터_거리
학교명             지역아동센터                       
경기초등학교          구세군지역아동복지센터        442.000864
                나무를심는학교            756.187664
                등대지역아동센터           732.545209
                신일지역아동센터           350.628703
                청소년1318해피존나무를심는학교  581.317492
...                                       ...
한신초등학교          즐거운지역아동센터          296.882035
한양초등학교          성동좋은이웃지역아동센터       634.295683
홍익대학교사범대학부속초등학교 망원무지개지역아동센터        821.639618
                옹달샘지역아동센터          822.461879
                행복무지개공부방지역아동센터     272.517655

[1687 rows x 1 columns]

In [208]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_local, columns = ['학교명','지역아동센터','지역아동센터_거리']) 
lscp_local_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_local_loc = pd.DataFrame(lscp_local_groupby.items())
df_local_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 지역아동센터 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_local_groupby.keys()):
  stations.append(lscp_local_groupby[k]['지역아동센터'].values)

#초등학교와 각 초등학교별 지역아동센터 데이터 프레임 합치기 
df_local_loc['지역아동센터'] = stations
df_local_loc = df_local_loc[[0, '지역아동센터']]
df_local_loc.columns = ['학교명', '지역아동센터']
df_local_loc

,학교명,지역아동센터
0,경기초등학교,"[등대지역아동센터, 나무를심는학교, 신일지역아동센터, 구세군지역아동복지센터, 청소년..."
1,경복초등학교,[행복한지역아동센터]
2,경희초등학교,"[해피아트지역아동센터, 행복한지역아동센터, 서울지역아동센터]"
3,광운초등학교,"[다솔지역아동센터, 장위지역아동센터밀알학교, 푸른하늘지역아동센터, 구립월계문화지역아..."
4,금성초등학교,[마루아라지역아동센터]
...,...,...
498,추계초등학교,"[나무를심는학교, 신일지역아동센터, 구세군지역아동복지센터, 청소년1318해피존나무를..."
499,충암초등학교,"[꿈이있는푸른학교지역아동센터, 비전스쿨지역아동센터, 광암지역아동센터, 새로핌지역아동센터]"
500,한신초등학교,"[쌍문지역아동센터, 방학동지역아동센터, 무지개지역아동센터, 에듀존도봉지역아동센터, ..."
501,한양초등학교,[성동좋은이웃지역아동센터]


In [271]:
df_local_loc = merge_base.merge(df_local_loc,on='학교명', how='outer')
df_local_loc

,학교명,지역아동센터
0,서울휘경초등학교,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터]"
1,서울동답초등학교,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
2,서울답십리초등학교,"[성동좋은이웃지역아동센터, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
3,서울신답초등학교,[조이스터디]
4,서울용두초등학교,[아름다운안암지역아동센터]
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,"[예지지역아동센터, 구로구공립지역아동센터]"
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


## 방과후아카데미

In [213]:
df_after = pd.read_excel('/content/drive/My Drive/Project/Seoul/초등학교/청소년방과후아카데미_초등부_200130기준.xlsx')
df_after

,연번,시도,시군구,시설명,운영유형,초등학생 수용인원(정원),소재지,Latitude,Longitude
0,1,서울,중구,유락종합사회복지관,기본지원형,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777
1,2,서울,중구,서울중구청소년수련관,기본지원형,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577
2,3,서울,성동구,시립성동청소년센터,기본지원형,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712
3,4,서울,동대문구,시립동대문청소년센터,기본지원형,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988
4,5,서울,중랑구,서울시립망우청소년센터,기본지원형,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848
5,6,서울,중랑구,시립중랑청소년센터,기본지원형,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839
6,7,서울,성북구,시립성북청소년센터(초등),기본지원형,40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734
7,8,서울,강북구,시립강북청소년센터,기본지원형,40,서울특별시 강북구 4.19로 74,37.645936,127.006380
8,9,서울,도봉구,"시립창동청소년센터(초등,중등)",기본지원형,20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615
9,10,서울,은평구,시립은평청소년센터,기본지원형,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524


#### 전처리

In [214]:
df_after = df_after.drop(['시도','연번', '운영유형'],axis = 1)
df_after

,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524


In [215]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [216]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(df_after['Longitude'], df_after['Latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_after = gpd.GeoDataFrame(df_after, crs=crs, geometry=geometry)
gdf_after

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777,POINT (127.02278 37.56505)
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577,POINT (127.01258 37.55178)
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306)
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727)
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588)
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317)
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757)
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594)
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827)
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415)


In [217]:
# 행정동 포함여부 확인 
gdf_after['행정동코드'] = np.zeros(len(gdf_after))
gdf_after['행정동'] = np.zeros(len(gdf_after))

for i in gdf_after.index:
    for j in dong_geo.index:
        if gdf_after.geometry[i].within(dong_geo.geometry[j]):
            gdf_after['행정동'][i] = dong_geo.읍면동명[j]
            gdf_after['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_after['행정동코드'] = gdf_after['행정동코드'].astype(int)
gdf_after

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,시군구,시설명,초등학생 수용인원(정원),소재지,Latitude,Longitude,geometry,행정동코드,행정동
0,중구,유락종합사회복지관,20,서울특별시 중구 퇴계로 460 7층,37.565054,127.022777,POINT (127.02278 37.56505),1114065000,신당제5동
1,중구,서울중구청소년수련관,20,서울특별시 중구 동호로 5길 19,37.551779,127.012577,POINT (127.01258 37.55178),1114063500,약수동
2,성동구,시립성동청소년센터,40,서울특별시 성동구 고산자로 260 2층,37.563055,127.036712,POINT (127.03671 37.56306),1120056000,행당제1동
3,동대문구,시립동대문청소년센터,40,서울특별시 동대문구 제기로 33길 25 3층,37.587269,127.049988,POINT (127.04999 37.58727),1123070500,청량리동
4,중랑구,서울시립망우청소년센터,20,서울특별시 중랑구 송림길 156 3층,37.605884,127.108848,POINT (127.10885 37.60588),1126065500,망우본동
5,중랑구,시립중랑청소년센터,60,서울특별시 중랑구 용마산로 217 4층,37.573171,127.085839,POINT (127.08584 37.57317),1126054000,면목제4동
6,성북구,시립성북청소년센터(초등),40,서울특별시 성북구 한천로 95길 7 3층,37.617570,127.055734,POINT (127.05573 37.61757),1129078000,장위제3동
7,강북구,시립강북청소년센터,40,서울특별시 강북구 4.19로 74,37.645936,127.006380,POINT (127.00638 37.64594),1130564500,우이동
8,도봉구,"시립창동청소년센터(초등,중등)",20,서울특별시 도봉구 노해로 69길 132(창동),37.658271,127.050615,POINT (127.05061 37.65827),1132051400,창제4동
9,은평구,시립은평청소년센터,40,서울특별시 은평구 백련산로 4길 16 1층,37.594149,126.924524,POINT (126.92452 37.59415),1138059000,응암제2동


In [ ]:
# gdf_element.to_file('/content/drive/My Drive/Project/Seoul/가공데이터/초등돌봄교실이용현황_위치포함.geojson', driver='GeoJSON')

### 800m 초등학교

In [218]:
lscp_after=pd.DataFrame(harv_dis(taregt_school,df_after,881), columns=['학교명','방과후아카데미','방과후아카데미_거리'])
lscp_after

,학교명,방과후아카데미,방과후아카데미_거리
0,서울휘경초등학교,시립동대문청소년센터,744.512486
1,서울홍릉초등학교,시립동대문청소년센터,430.109136
2,서울청량초등학교,시립동대문청소년센터,765.678122
3,경희초등학교,시립동대문청소년센터,859.344208
4,서울삼육초등학교,시립동대문청소년센터,298.955468
...,...,...,...
62,서울장곡초등학교,시립성북청소년센터(초등),672.468199
63,광운초등학교,시립성북청소년센터(초등),174.876372
64,서울인수초등학교,시립강북청소년센터,402.239741
65,서울강솔초등학교,시립강동청소년센터,811.662989


In [220]:
# 방과후아카데미 하나에 여러 학교 
lscp_after.groupby(['방과후아카데미','학교명']).agg('mean')

방과후아카데미_거리
방과후아카데미       학교명                 
마포구립망원청소년문화센터 서울동교초등학교  372.337711
              서울망원초등학교  470.597189
서울시립망우청소년센터   서울동원초등학교  410.406832
              서울신내초등학교  775.259464
서울중구청소년수련관    동산초등학교    613.242766
...                            ...
유락종합사회복지관     서울금북초등학교  842.966503
              서울무학초등학교  634.191531
              서울숭신초등학교  545.572611
              서울신당초등학교  177.417227
              서울흥인초등학교  861.232840

[67 rows x 1 columns]

In [221]:
# 학교 하나에 여러 방과후아카데미 
lscp_after.groupby(['학교명', '방과후아카데미']).agg('mean')

,,방과후아카데미_거리
학교명,방과후아카데미,
경희초등학교,시립동대문청소년센터,859.344208
광운초등학교,시립성북청소년센터(초등),174.876372
동산초등학교,서울중구청소년수련관,613.242766
서울가인초등학교,"시립창동청소년센터(초등,중등)",687.491941
서울강솔초등학교,시립강동청소년센터,811.662989
...,...,...
서울행당초등학교,시립성동청소년센터,690.454404
서울행현초등학교,시립성동청소년센터,878.935237
서울홍릉초등학교,시립동대문청소년센터,430.109136


In [222]:
#초등학교별로 groupby 
df_loc = pd.DataFrame(lscp_after, columns = ['학교명','방과후아카데미','방과후아카데미_거리']) 
lscp_after_groupby = dict(list(df_loc.groupby('학교명')))

# 전체 초등학교 리스트를 데이터 프레임으로 만들기 
df_after_loc = pd.DataFrame(lscp_after_groupby.items())
df_after_loc['학교명'] = np.array(np.nan)

# 각 초등학교의 가까운 방과후아카데미 리스트를 데이터 프레임으로 만들기 
stations = []
for idx, k in enumerate(lscp_after_groupby.keys()):
  stations.append(lscp_after_groupby[k]['방과후아카데미'].values)

#초등학교와 각 초등학교별 방과후아카데미 데이터 프레임 합치기 
df_after_loc['방과후아카데미'] = stations
df_after_loc = df_after_loc[[0, '방과후아카데미']]
df_after_loc.columns = ['학교명', '방과후아카데미']
df_after_loc

,학교명,방과후아카데미
0,경희초등학교,[시립동대문청소년센터]
1,광운초등학교,[시립성북청소년센터(초등)]
2,동산초등학교,[서울중구청소년수련관]
3,서울가인초등학교,"[시립창동청소년센터(초등,중등)]"
4,서울강솔초등학교,[시립강동청소년센터]
...,...,...
59,서울행당초등학교,[시립성동청소년센터]
60,서울행현초등학교,[시립성동청소년센터]
61,서울홍릉초등학교,[시립동대문청소년센터]
62,서울휘경초등학교,[시립동대문청소년센터]


In [224]:
df_after_loc = merge_base.merge(df_after_loc,on='학교명', how='outer')
df_after_loc

,학교명,방과후아카데미
0,서울휘경초등학교,[시립동대문청소년센터]
1,서울동답초등학교,NaN
2,서울답십리초등학교,NaN
3,서울신답초등학교,NaN
4,서울용두초등학교,NaN
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,NaN
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


## Merge

In [279]:
df_after_loc

,학교명,방과후아카데미
0,서울휘경초등학교,[시립동대문청소년센터]
1,서울동답초등학교,NaN
2,서울답십리초등학교,NaN
3,서울신답초등학교,NaN
4,서울용두초등학교,NaN
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,NaN
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


In [280]:
df_local_loc

,학교명,지역아동센터
0,서울휘경초등학교,"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터]"
1,서울동답초등학교,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
2,서울답십리초등학교,"[성동좋은이웃지역아동센터, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
3,서울신답초등학교,[조이스터디]
4,서울용두초등학교,[아름다운안암지역아동센터]
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,"[예지지역아동센터, 구로구공립지역아동센터]"
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


In [281]:
gdf_element

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824)


In [282]:
df_dolbom_loc

,학교명,돌봄센터
0,서울휘경초등학교,NaN
1,서울동답초등학교,[동대문2호점]
2,서울답십리초등학교,NaN
3,서울신답초등학교,"[성동8호점, 성동3호점]"
4,서울용두초등학교,"[성동8호점, 동대문1호점]"
...,...,...
598,서울위례별초등학교,NaN
599,서울하늘숲초등학교,[구로5호점]
600,서울언남초등학교,NaN
601,서울항동초등학교,NaN


In [286]:
df_1 = gdf_element.merge(df_dolbom_loc, on='학교명')
df_2 = df_1.merge(df_after_loc, on='학교명')
df_3 = df_2.merge(df_local_loc, on='학교명')

df_3

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,위도,경도,geometry,돌봄센터,방과후아카데미,지역아동센터
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,37.585736,127.058213,POINT (37.586 127.058),NaN,[시립동대문청소년센터],"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터]"
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,37.571503,127.064034,POINT (37.572 127.064),[동대문2호점],NaN,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,37.568804,127.055429,POINT (37.569 127.055),NaN,NaN,"[성동좋은이웃지역아동센터, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]"
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,37.572705,127.042962,POINT (37.573 127.043),"[성동8호점, 성동3호점]",NaN,[조이스터디]
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,37.576415,127.028387,POINT (37.576 127.028),"[성동8호점, 동대문1호점]",NaN,[아름다운안암지역아동센터]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,37.479086,127.145042,POINT (37.479 127.145),NaN,NaN,NaN
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,37.483405,126.842135,POINT (37.483 126.842),[구로5호점],NaN,"[예지지역아동센터, 구로구공립지역아동센터]"
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,37.453898,127.061885,POINT (37.454 127.062),NaN,NaN,NaN
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,37.476972,126.824078,POINT (37.477 126.824),NaN,NaN,NaN


In [290]:
df_final = df_3[['행정동코드', '구', '행정동', '학교명', '소재지도로명주소', '돌봄교실_개수', '돌봄교실_이용인원','돌봄센터', '방과후아카데미', '지역아동센터', '위도',
       '경도', 'geometry']]
df_final

,행정동코드,구,행정동,학교명,소재지도로명주소,돌봄교실_개수,돌봄교실_이용인원,돌봄센터,방과후아카데미,지역아동센터,위도,경도,geometry
0,1123073000,동대문구,휘경제2동,서울휘경초등학교,서울특별시 동대문구 망우로6길 48,3.0,72.0,NaN,[시립동대문청소년센터],"[염광지역아동센터빛나는집, 해피아트지역아동센터, 원광지역아동센터, 나누리지역아동센터]",37.585736,127.058213,POINT (37.586 127.058)
1,1123061000,동대문구,답십리제2동,서울동답초등학교,서울특별시 동대문구 답십리로60길 12,4.0,93.0,[동대문2호점],NaN,"[염광지역아동센터빛나는집, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",37.571503,127.064034,POINT (37.572 127.064)
2,1123060000,동대문구,답십리제1동,서울답십리초등학교,서울특별시 동대문구 전농로3길 23,4.0,89.0,NaN,NaN,"[성동좋은이웃지역아동센터, 장안지역아동센터, 꿈사랑방지역아동센터, 구립동대문지역아동센터]",37.568804,127.055429,POINT (37.569 127.055)
3,1123060000,동대문구,답십리제1동,서울신답초등학교,서울특별시 동대문구 천호대로 177,4.0,92.0,"[성동8호점, 성동3호점]",NaN,[조이스터디],37.572705,127.042962,POINT (37.573 127.043)
4,1123053600,동대문구,용신동,서울용두초등학교,서울특별시 동대문구 천호대로13길 43,4.0,73.0,"[성동8호점, 동대문1호점]",NaN,[아름다운안암지역아동센터],37.576415,127.028387,POINT (37.576 127.028)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,1171064700,송파구,위례동,서울위례별초등학교,서울특별시 송파구 위례광장로 243,7.0,121.0,NaN,NaN,NaN,37.479086,127.145042,POINT (37.479 127.145)
599,1153078000,구로구,오류제2동,서울하늘숲초등학교,서울특별시 구로구 오리로 1100,3.0,75.0,[구로5호점],NaN,"[예지지역아동센터, 구로구공립지역아동센터]",37.483405,126.842135,POINT (37.483 126.842)
600,1165066000,서초구,내곡동,서울언남초등학교,서울특별시 서초구 헌릉로8길 42,7.0,174.0,NaN,NaN,NaN,37.453898,127.061885,POINT (37.454 127.062)
601,1153080000,구로구,항동,서울항동초등학교,서울특별시 구로구 연동로 178,4.0,45.0,NaN,NaN,NaN,37.476972,126.824078,POINT (37.477 126.824)


In [291]:
# geojson은 numpy가 들어갈수없다고 합니다...
# pickle로 일단 만들었습니다. 
df_final.to_pickle('/content/drive/My Drive/Project/Seoul/가공데이터/학교별_주변공적돌봄시설.pickle')